<a href="https://colab.research.google.com/github/sidneygehring/CampusPoliceOptimization/blob/main/Final_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Team: Stella Shipman, Kailey Phillips, Sidney Gehring

In [ ]:
# Install pyomo
!pip install -q pyomo

In [ ]:
# Install Solvers
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [ ]:
# Import packages
import numpy as np
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import folium

In [ ]:
# Create the model
model = pyo.ConcreteModel()

In [ ]:
# Load in 'node_distances' and 'final_summary' file
from google.colab import files
files.upload()

Saving final_summary.csv to final_summary.csv
Saving node_distances.csv to node_distances.csv


{'final_summary.csv': b'cluster,avg_crime_score,incident_count,crime_weight,normalized_weight,lat,lon\n0,6.0,2,12.0,0.042553191489361694,36.07233,-94.16925\n1,6.625,8,53.0,0.26063829787234044,36.070135,-94.17239\n2,4.571428571428571,42,192.0,0.9999999999999999,36.072088292857146,-94.17507849612734\n3,6.5,2,13.0,0.047872340425531915,36.05908,-94.18025\n4,4.857142857142857,7,34.0,0.15957446808510636,36.06599571428571,-94.16871142857144\n5,2.2857142857142856,7,16.0,0.06382978723404255,36.06162,-94.17555857142858\n6,7.25,4,29.0,0.13297872340425532,36.06073000000001,-94.1790475\n7,5.333333333333333,3,16.0,0.06382978723404255,36.05893333333333,-94.18374333333334\n8,6.0,2,12.0,0.042553191489361694,36.06055,-94.17619\n9,4.533333333333333,15,68.0,0.3404255319148936,36.071320666666665,-94.17050133333332\n10,4.857142857142857,7,34.0,0.15957446808510636,36.06775097142857,-94.1760535759671\n11,3.4482758620689653,29,100.0,0.5106382978723404,36.06559137931035,-94.17510586206896\n12,4.384615384615385,

In [ ]:
# Load files into df
locations = pd.read_csv('final_summary.csv', header = 0)
distances = pd.read_csv('node_distances.csv', header = 0)

In [ ]:
# Round distances and convert data types
distances = pd.read_csv('node_distances.csv')
distances['distance_meters'] = distances['distance_meters'].round(2)
distances['cluster_id_1'] = distances['cluster_id_1'].astype(int)
distances['cluster_id_2'] = distances['cluster_id_2'].astype(int)

distances.head()

,cluster_id_1,cluster_id_2,distance_meters
0,0,1,373.27
1,0,2,525.72
2,0,3,1773.02
3,0,4,704.53
4,0,5,1317.29


In [ ]:
# Add 1 to weights to prevent multiplication by zero in the objective
# Convert data types
locations['normalized_weight'] = locations['normalized_weight'] +1
locations['cluster'] = locations['cluster'].astype(int)

locations.head()

,cluster,avg_crime_score,incident_count,crime_weight,normalized_weight,lat,lon
0,0,6.000000,2,12.0,1.042553,36.072330,-94.169250
1,1,6.625000,8,53.0,1.260638,36.070135,-94.172390
2,2,4.571429,42,192.0,2.000000,36.072088,-94.175078
3,3,6.500000,2,13.0,1.047872,36.059080,-94.180250
4,4,4.857143,7,34.0,1.159574,36.065996,-94.168711


In [ ]:
# Incident locations and possible officer stations both come from cluster list
incident_locations = locations['cluster'].tolist()
station_locations = locations['cluster'].tolist()

# Crime scores come from the normalized crime weights
crime_scores = locations['normalized_weight'].tolist()

# Dictionary that lists the distance between each node combination
distance_m = {
    (row.cluster_id_1, row.cluster_id_2): row.distance_meters
    for row in distances.itertuples(index=False)
}

# Make distance_m symmetric to make sure it matches the (i,j) node pairs
symmetric_distance_m = {}
for (i, j), dist in distance_m.items():
    symmetric_distance_m[(i, j)] = dist
    symmetric_distance_m[(j, i)] = dist  # Add the reversed pair

distance_m = symmetric_distance_m

# Fill in any missing (i, j) pairs where i = j
all_pairs = [(i, j) for i in incident_locations for j in station_locations]

for i, j in all_pairs:
    if i == j:
        # Inputting 1 to prevent multiplying by zero in objective function
        distance_m[(i, j)] = 1

# Officers number comes from info gathered from UAPD security officer
officers = 4

In [ ]:
# Sets
model.I = pyo.Set(initialize = incident_locations)  # Set of incident locations
model.J = pyo.Set(initialize = station_locations)   # Set of potential station locations

In [ ]:
# Parameters
model.d = pyo.Param(model.I, model.J, initialize = distance_m, default = float('inf')) # Distance
model.p = pyo.Param(initialize = officers)      # Number of officers to station
model.w = pyo.Param(model.I, initialize = crime_scores)  # Crime scores for each location


In [ ]:
# Decision Variables
model.x = pyo.Var(model.J, domain = pyo.Binary)     # 1 if an officer is stationed at j, 0 otherwise
model.y = pyo.Var(model.I, model.J, domain = pyo.Binary)  # 1 if location i is served by station j


In [ ]:
# Objective Function: Minimize total response time
def obj_rule(model):
    return sum(model.w[i] * model.d[i, j] * model.y[i, j] for i in model.I for j in model.J)

model.OBJ = pyo.Objective(rule = obj_rule, sense = pyo.minimize)

In [ ]:
# Each location must be assigned to exactly one station
def single_assignment_rule(model, i):
    return sum(model.y[i, j] for j in model.J) == 1

model.SingleAssignment = pyo.Constraint(model.I, rule = single_assignment_rule)


In [ ]:
# Can only assign locations to stations where officers are located
def station_coverage_rule(model, i, j):
      return model.y[i, j] <= model.x[j]

model.StationCoverage = pyo.Constraint(model.I, model.J, rule = station_coverage_rule)


In [ ]:
# Limit on number of officers/stations
def officer_limit_rule(model):
    return sum(model.x[j] for j in model.J) == model.p

model.OfficerLimit = pyo.Constraint(rule=officer_limit_rule)


In [ ]:
# Solve the model
solver = pyo.SolverFactory('cbc')
results = solver.solve(model, tee=True)


Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmplslkcvf0.pyomo.lp -stat=1 -solve -solu /tmp/tmplslkcvf0.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 2757 (0) rows, 2756 (0) columns and 8164 (0) elements
Statistics for presolved model
Original problem has 2756 integers (2756 of which binary)
==== 52 zero objective 2658 different
==== absolute objective values 2658 different
==== for integers 52 zero objective 2658 different
==== for integers absolute objective values 2658 different
===== end objective counts


Problem has 2757 rows, 2756 columns (2704 with objective) and 8164 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 2756 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 52 of type E 1.0, 0 of type E -1.0, 


In [ ]:
# Process results
if results.solver.status == pyo.SolverStatus.ok:
    # Get selected stations
    selected_stations = [j for j in model.J if pyo.value(model.x[j]) > 0.5]

    # Get assignments
    assignments = {i: [j for j in model.J if pyo.value(model.y[i, j]) > 0.5][0] for i in model.I}

    # Get objective value
    objective_value = pyo.value(model.OBJ)

    print(selected_stations, assignments, objective_value)
else:
    print("Solver did not find an optimal solution.")

[2, 15, 31, 33] {0: 2, 1: 2, 2: 2, 3: 15, 4: 33, 5: 15, 6: 15, 7: 15, 8: 15, 9: 2, 10: 2, 11: 33, 12: 2, 13: 2, 14: 2, 15: 15, 16: 33, 17: 2, 18: 15, 19: 31, 20: 33, 21: 33, 22: 2, 23: 33, 24: 15, 25: 33, 26: 15, 27: 2, 28: 15, 29: 15, 30: 2, 31: 31, 32: 31, 33: 33, 34: 2, 35: 2, 36: 33, 37: 2, 38: 2, 39: 15, 40: 2, 41: 33, 42: 2, 43: 2, 44: 33, 45: 2, 46: 15, 47: 2, 48: 15, 49: 2, 50: 33, 51: 15} 22833.14186170213


In [ ]:
# Extract data for map
# Get stationed officer locations
stationed = [j for j in model.J if pyo.value(model.x[j]) > 0.5]

# Create incident-to-station assignments
assignments = []
for i in model.I:
    for j in model.J:
        if pyo.value(model.y[i, j]) > 0.5:
            assignments.append((i, j))


In [ ]:
# Create a quick lookup for lat/lon
loc_dict = locations.set_index('cluster')[['lat', 'lon']].to_dict('index')


In [ ]:
# Start map centered at campus
center_lat = locations['lat'].mean()
center_lon = locations['lon'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Plot stationed officer locations
for j in stationed:
    coords = loc_dict[j]
    folium.Marker(
        location=[coords['lat'], coords['lon']],
        icon=folium.Icon(color='red', icon='shield', prefix='fa'),
        popup=f"Officer Station {j}"
    ).add_to(m)

# Plot incident locations and lines to stations
for i, j in assignments:
    i_coords = loc_dict[i]
    j_coords = loc_dict[j]

    # Add location marker
    folium.CircleMarker(
        location=[i_coords['lat'], i_coords['lon']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=f"Incident {i} assigned to Station {j}"
    ).add_to(m)

    # Optional: add line from location to station
    folium.PolyLine(
        locations=[
            [i_coords['lat'], i_coords['lon']],
            [j_coords['lat'], j_coords['lon']]
        ],
        color='gray',
        weight=1,
        opacity=0.5
    ).add_to(m)


In [ ]:
# Display Map
m